In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector, state_fidelity

In [ ]:

# Generate a random quantum circuit
def generate_random_circuit(num_qubits, depth):
    circuit = QuantumCircuit(num_qubits)
    for _ in range(depth):
        for qubit in range(num_qubits):
            circuit.rx(np.random.uniform(0, 2 * np.pi), qubit)
            circuit.ry(np.random.uniform(0, 2 * np.pi), qubit)
            circuit.rz(np.random.uniform(0, 2 * np.pi), qubit)
        for qubit in range(num_qubits - 1):
            circuit.cz(qubit, qubit + 1)
    return circuit

In [ ]:
# Perform randomized benchmarking
def randomized_benchmarking(num_qubits, depths, num_sequences, shots):
    backend = Aer.get_backend('aer_simulator')  # AerSimulator backend
    results = []
    for depth in depths:
        success_counts = 0
        for _ in range(num_sequences):
            # Generate a random circuit and its inverse
            circuit = generate_random_circuit(num_qubits, depth)
            inverse_circuit = circuit.inverse()

            # Transpile the circuits for the backend
            transpiled_circuit = transpile(circuit, backend)
            transpiled_inverse_circuit = transpile(inverse_circuit, backend)

            # Apply the circuit and obtain the final statevector
            transpiled_circuit.save_statevector()  # Ensure statevector is saved
            circuit_result = backend.run(transpiled_circuit).result()  # Run the transpiled circuit
            final_statevector = Statevector(circuit_result.get_statevector())

            # Apply the inverse circuit and obtain the final statevector
            transpiled_inverse_circuit.save_statevector()  # Ensure statevector is saved
            inverse_result = backend.run(transpiled_inverse_circuit).result()  # Run the transpiled inverse circuit
            inverse_statevector = Statevector(inverse_result.get_statevector())

            # Calculate the fidelity
            fidelity = state_fidelity(final_statevector, inverse_statevector)
            success_counts += shots * (1 - fidelity)

        success_rate = success_counts / (num_sequences * shots)
        results.append(success_rate)
    return results

# Example usage
num_qubits = 2
depths = [1, 2, 3, 4]
num_sequences = 100
shots = 1024

results = randomized_benchmarking(num_qubits, depths, num_sequences, shots)
print(results)